In [1]:
import javabridge
import os
import glob
import pandas as pd
import pydot
from IPython.display import SVG

In [2]:
tetrad_libdir = os.path.join(os.getcwd(), '../../','src', 'pycausal', 'lib')
for l in glob.glob(tetrad_libdir + os.sep + "*.jar"):
    print l
    javabridge.JARS.append(str(l))

/home/jovyan/py-causal/example/javabridge/../../src/pycausal/lib/causal-cmd-0.2.0-SNAPSHOT.jar
/home/jovyan/py-causal/example/javabridge/../../src/pycausal/lib/tetrad-lib-6.4.0-SNAPSHOT-tetradcmd.jar
/home/jovyan/py-causal/example/javabridge/../../src/pycausal/lib/data-reader-0.2.0-SNAPSHOT.jar


In [3]:
javabridge.start_vm(run_headless=True, max_heap_size = '100M')
javabridge.attach()

In [4]:
data_dir = os.path.join(os.getcwd(), '../../','data', 'audiology.txt')
data_dir

'/home/jovyan/py-causal/example/javabridge/../../data/audiology.txt'

In [5]:
dframe = pd.read_table(data_dir, sep="\t")
dframe.head()

,age_gt_60,air,airBoneGap,ar_c,ar_u,bone,boneAbnormal,bser,history_buzzing,history_dizziness,...,s_sn_gt_2k,s_sn_gt_4k,speech,static_normal,tymp,viith_nerve_signs,wave_V_delayed,waveform_ItoV_prolonged,indentifier,class
0,f,mild,f,normal,normal,?,t,?,f,f,...,f,f,normal,t,a,f,f,f,p1,cochlear_unknown
1,f,moderate,f,normal,normal,?,t,?,f,f,...,f,f,normal,t,a,f,f,f,p2,cochlear_unknown
2,t,mild,t,?,absent,mild,t,?,f,f,...,f,f,normal,t,as,f,f,f,p3,mixed_cochlear_age_fixation
3,t,mild,t,?,absent,mild,f,?,f,f,...,f,f,normal,t,b,f,f,f,p4,mixed_cochlear_age_otitis_media
4,t,mild,f,normal,normal,mild,t,?,f,f,...,f,f,good,t,a,f,f,f,p5,cochlear_age


In [6]:
dataBox = javabridge.JClassWrapper("edu.cmu.tetrad.data.VerticalIntDataBox")(len(dframe.index),dframe.columns.size)

In [7]:
node_list = javabridge.JClassWrapper("java.util.ArrayList")()
# load dataset
col_no = 0
for col in dframe.columns:
    
    cat_array = sorted(set(dframe[col]))
    cat_list = javabridge.JClassWrapper("java.util.ArrayList")()

    for cat in cat_array:
        cat = str(cat)
        cat_list.add(cat)
        
    nodi = javabridge.JClassWrapper("edu.cmu.tetrad.data.DiscreteVariable")(col, cat_list)
    node_list.add(nodi)
    
    for row in dframe.index:
        value = javabridge.JClassWrapper("java.lang.Integer") (cat_array.index(dframe.ix[row][col_no]))
        dataBox.set(row,col_no,value)
    
    col_no = col_no + 1

In [8]:
boxData = javabridge.JClassWrapper("edu.cmu.tetrad.data.BoxDataSet")(dataBox, node_list)

In [9]:
alpha = 0.05
indTest = javabridge.JClassWrapper('edu.cmu.tetrad.search.IndTestChiSquare')(boxData, alpha)

In [10]:
rfci = javabridge.JClassWrapper('edu.cmu.tetrad.search.Rfci')(indTest)

In [11]:
rfci.setDepth(-1)
rfci.setVerbose(True)

In [12]:
prior = javabridge.JClassWrapper('edu.cmu.tetrad.data.Knowledge2')()
prior.setForbidden('history_noise','class') # forbidden directed edges
prior.setForbidden('history_fluctuating','class') # forbidden directed edges
prior.setTierForbiddenWithin(0, True)
prior.addToTier(0, 'class')
prior.addToTier(0, 'history_fluctuating')
prior.addToTier(0, 'history_noise')
rfci.setKnowledge(prior)
prior

Instance of edu.cmu.tetrad.data.Knowledge2: /knowledge
addtemporal

0* class history_fluctuating history_noise 

forbiddirect

history_fluctuating ==> class 
history_noise ==> class 
requiredirect


In [13]:
tetradGraph = rfci.search()
tetradGraph

Instance of edu.cmu.tetrad.graph.EdgeListGraphSingleConnections: Graph Nodes:
age_gt_60,air,airBoneGap,ar_c,ar_u,bone,boneAbnormal,bser,history_buzzing,history_dizziness,history_fluctuating,history_fullness,history_heredity,history_nausea,history_noise,history_recruitment,history_ringing,history_roaring,history_vomiting,late_wave_poor,m_at_2k,m_cond_lt_1k,m_gt_1k,m_m_gt_2k,m_m_sn,m_m_sn_gt_1k,m_m_sn_gt_2k,m_m_sn_gt_500,m_p_sn_gt_2k,m_s_gt_500,m_s_sn,m_s_sn_gt_1k,m_s_sn_gt_2k,m_s_sn_gt_3k,m_s_sn_gt_4k,m_sn_2_3k,m_sn_gt_1k,m_sn_gt_2k,m_sn_gt_3k,m_sn_gt_4k,m_sn_gt_500,m_sn_gt_6k,m_sn_lt_1k,m_sn_lt_2k,m_sn_lt_3k,middle_wave_poor,mod_gt_4k,mod_mixed,mod_s_mixed,mod_s_sn_gt_500,mod_sn,mod_sn_gt_1k,mod_sn_gt_2k,mod_sn_gt_3k,mod_sn_gt_4k,mod_sn_gt_500,notch_4k,notch_at_4k,o_ar_c,o_ar_u,s_sn_gt_1k,s_sn_gt_2k,s_sn_gt_4k,speech,static_normal,tymp,viith_nerve_signs,wave_V_delayed,waveform_ItoV_prolonged,indentifier,class

Graph Edges:
1. age_gt_60 o-> class
2. air <-> class
3. ar_c o-> o_ar_u
4. a

In [14]:
tetradGraph.toString()

u'Graph Nodes:\nage_gt_60,air,airBoneGap,ar_c,ar_u,bone,boneAbnormal,bser,history_buzzing,history_dizziness,history_fluctuating,history_fullness,history_heredity,history_nausea,history_noise,history_recruitment,history_ringing,history_roaring,history_vomiting,late_wave_poor,m_at_2k,m_cond_lt_1k,m_gt_1k,m_m_gt_2k,m_m_sn,m_m_sn_gt_1k,m_m_sn_gt_2k,m_m_sn_gt_500,m_p_sn_gt_2k,m_s_gt_500,m_s_sn,m_s_sn_gt_1k,m_s_sn_gt_2k,m_s_sn_gt_3k,m_s_sn_gt_4k,m_sn_2_3k,m_sn_gt_1k,m_sn_gt_2k,m_sn_gt_3k,m_sn_gt_4k,m_sn_gt_500,m_sn_gt_6k,m_sn_lt_1k,m_sn_lt_2k,m_sn_lt_3k,middle_wave_poor,mod_gt_4k,mod_mixed,mod_s_mixed,mod_s_sn_gt_500,mod_sn,mod_sn_gt_1k,mod_sn_gt_2k,mod_sn_gt_3k,mod_sn_gt_4k,mod_sn_gt_500,notch_4k,notch_at_4k,o_ar_c,o_ar_u,s_sn_gt_1k,s_sn_gt_2k,s_sn_gt_4k,speech,static_normal,tymp,viith_nerve_signs,wave_V_delayed,waveform_ItoV_prolonged,indentifier,class\n\nGraph Edges:\n1. age_gt_60 o-> class\n2. air <-> class\n3. ar_c o-> o_ar_u\n4. ar_u <-> o_ar_u\n5. bone o-o air\n6. bone <-> class\n7. b

In [15]:
tetradGraph.getNodeNames()

Instance of java.util.ArrayList: [age_gt_60, air, airBoneGap, ar_c, ar_u, bone, boneAbnormal, bser, history_buzzing, history_dizziness, history_fluctuating, history_fullness, history_heredity, history_nausea, history_noise, history_recruitment, history_ringing, history_roaring, history_vomiting, late_wave_poor, m_at_2k, m_cond_lt_1k, m_gt_1k, m_m_gt_2k, m_m_sn, m_m_sn_gt_1k, m_m_sn_gt_2k, m_m_sn_gt_500, m_p_sn_gt_2k, m_s_gt_500, m_s_sn, m_s_sn_gt_1k, m_s_sn_gt_2k, m_s_sn_gt_3k, m_s_sn_gt_4k, m_sn_2_3k, m_sn_gt_1k, m_sn_gt_2k, m_sn_gt_3k, m_sn_gt_4k, m_sn_gt_500, m_sn_gt_6k, m_sn_lt_1k, m_sn_lt_2k, m_sn_lt_3k, middle_wave_poor, mod_gt_4k, mod_mixed, mod_s_mixed, mod_s_sn_gt_500, mod_sn, mod_sn_gt_1k, mod_sn_gt_2k, mod_sn_gt_3k, mod_sn_gt_4k, mod_sn_gt_500, notch_4k, notch_at_4k, o_ar_c, o_ar_u, s_sn_gt_1k, s_sn_gt_2k, s_sn_gt_4k, speech, static_normal, tymp, viith_nerve_signs, wave_V_delayed, waveform_ItoV_prolonged, indentifier, class]

In [16]:
tetradGraph.getEdges()

Instance of java.util.HashSet: [m_sn_gt_2k o-o bser, history_roaring o-o history_nausea, m_sn_gt_1k o-o history_recruitment, boneAbnormal o-> air, notch_4k o-o m_m_sn_gt_2k, o_ar_c o-> ar_u, ar_c o-> o_ar_u, bone <-> class, age_gt_60 o-> class, speech o-> air, bser o-> class, ar_u <-> o_ar_u, m_sn_gt_500 o-o history_ringing, notch_at_4k o-> class, tymp o-> class, bone o-o air, boneAbnormal o-> bone, speech o-> bone, history_nausea o-o history_dizziness, air <-> class]

In [17]:
javabridge.detach()
javabridge.kill_vm()